# Adding particles using NASA JPL Horizons system

REBOUND can add particles to simulations by obtaining ephemerides from NASA's powerful HORIZONS database.  HORIZONS supports many different options, and we will certainly not try to cover everything here.  This is meant to serve as an introduction to the basics, beyond what's in [Churyumov-Gerasimenko.ipynb](Churyumov-Gerasimenko.ipynb).  If you catch any errors, or would either like to expand on this documentation or improve REBOUND's HORIZONS interface (`rebound/horizons.py`), please do fork the repository and send us a pull request.

**Adding particles**

When we add particles by passing a string, REBOUND queries the HORIZONS database and takes the first dataset HORIZONS offers.  For the Sun, moons, and small bodies, this will typically return the body itself.  For planets, it will return the barycenter of the system (for moonless planets like Venus it will say barycenter but there is no distinction).  If you want the planet specifically, you have to use, e.g., "NAME=Pluto" rather than "Pluto".  In all cases, REBOUND will print out the name of the HORIZONS entry it's using.

You can also add bodies using their integer NAIF IDs:  [NAIF IDs](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/MATLAB/req/naif_ids.html).  Note that because of the number of small bodies (asteroids etc.) we have discovered, this convention only works for large objects.  For small bodies, instead use "NAME=name" (see the SMALL BODIES section in the [HORIZONS Documentation](https://ssd.jpl.nasa.gov/?horizons_doc)).

In [1]:
import rebound
sim = rebound.Simulation()
sim.add("Sun")
## Other examples:
# sim.add("Venus")
# sim.add("399")
# sim.add("Europa")
# sim.add("NAME=Ida")
# sim.add("Pluto")
# sim.add("NAME=Pluto")
sim.status()

Searching NASA Horizons for 'Sun'... Found: Sun (10).
---------------------------------
REBOUND version:     	2.6.0
REBOUND built on:    	Aug 26 2015 08:10:34
Number of particles: 	1
Selected integrator: 	ias15
Simulation time:     	0.000000
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, id=-1 m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
---------------------------------


Currently, HORIZONS does not have any mass information for solar system bodies.  `rebound/horizons.py` has a hard-coded list provided by Jon Giorgini (10 May 2015) that includes the planets, their barycenters (total mass of planet plus moons), and the largest moons.  If REBOUND doesn't find the corresponding mass for an object from this list (like for the asteroid Ida above), it will print a warning message.  If you need the body's mass for your simulation, you can set it manually, e.g. (see [Units.ipynb](Units.ipynb) for an overview of using different units):

In [2]:
sim.add("NAME=Ida")
print(sim.particles[-1])    # Ida before setting the mass
sim.particles[-1].m = 2.1e-14 # Setting mass of Ida in Solar masses
print(sim.particles[-1])    # Ida after setting the mass

Searching NASA Horizons for 'NAME=Ida'... Found: 243 Ida.
<rebound.Particle object, id=-1 m=0.0 x=2.04968736319 y=1.84863060188 z=0.0533595378052 vx=-0.419860793519 vy=0.445752362086 vz=0.00225391332774>
<rebound.Particle object, id=-1 m=2.1e-14 x=2.04968736319 y=1.84863060188 z=0.0533595378052 vx=-0.419860793519 vy=0.445752362086 vz=0.00225391332774>


**Time**

By default, REBOUND queries HORIZONS for objects' current positions.  Specifically, it caches the current time the first time you call `rebound.add`, and gets the corresponding ephemeris.  All subsequent calls to `rebound.add` will then use that initial cached time to make sure you get a synchronized set of ephemerides.

You can also explicitly pass REBOUND the time at which you would like the particles ephemerides:  

In [3]:
sim = rebound.Simulation()
date = "2005-06-30 15:24" # You can also use Julian Days. For example:   date = "JD2458327.500000"
sim.add("Venus")
sim.add("Venus", date=date)
sim.status()

Searching NASA Horizons for 'Venus'... Found: Venus Barycenter (299).
Searching NASA Horizons for 'Venus'... Found: Venus Barycenter (299).
---------------------------------
REBOUND version:     	2.6.0
REBOUND built on:    	Aug 26 2015 08:10:34
Number of particles: 	2
Selected integrator: 	ias15
Simulation time:     	0.000000
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, id=-1 m=2.44783828778e-06 x=0.680209466768 y=-0.254854093348 z=-0.0427459769103 vx=0.406121578792 vy=1.09596198751 vz=-0.00841198680693>
<rebound.Particle object, id=-1 m=2.44783828778e-06 x=-0.665484474999 y=0.268717588536 z=0.0420869454006 vx=-0.445600653913 vy=-1.09579691343 vz=0.0107281126045>
---------------------------------


We see that the two Venus positions are different.  The first call cached the current time, but since the second call specified a date, it overrode the default.  Any time you pass a date, it will overwrite the default cached time, so: 

In [4]:
sim = rebound.Simulation()
date = "2005-06-30 15:24"   # You can also use Julian Days. For example:   date = "JD2458327.500000"
sim.add("Venus", date=date)
sim.add("Earth")
sim.status()

Searching NASA Horizons for 'Venus'... Found: Venus Barycenter (299).
Searching NASA Horizons for 'Earth'... Found: Earth-Moon Barycenter (3).
---------------------------------
REBOUND version:     	2.6.0
REBOUND built on:    	Aug 26 2015 08:10:34
Number of particles: 	2
Selected integrator: 	ias15
Simulation time:     	0.000000
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, id=-1 m=2.44783828778e-06 x=-0.665484474999 y=0.268717588536 z=0.0420869454006 vx=-0.445600653913 vy=-1.09579691343 vz=0.0107281126045>
<rebound.Particle object, id=-1 m=3.04043264802e-06 x=0.898697621871 y=-0.462254753275 z=1.20294531183e-05 vx=0.441185307329 vy=0.885643863577 vz=-3.27489122994e-05>
---------------------------------


would set up a simulation with Venus and Earth, all synchronized to 2005-06-30 15:24.  All dates should either be passed in the format Year-Month-Day Hour:Minute or in JDxxxxxxx.xxxx for a date in Julian Days.

REBOUND takes these absolute times to the nearest minute, since at the level of seconds you have to worry about exactly what time system you're using, and small additional perturbations probably start to matter.  For reference HORIZONS interprets all times for ephemerides as [Coordinate (or Barycentric Dynamical) Time](https://en.wikipedia.org/wiki/Barycentric_Dynamical_Time).

**Reference Frame**

REBOUND queries for particles' positions and velocities relative to the Sun:

In [5]:
sim = rebound.Simulation()
sim.add("Sun")
sim.status()

Searching NASA Horizons for 'Sun'... Found: Sun (10).
---------------------------------
REBOUND version:     	2.6.0
REBOUND built on:    	Aug 26 2015 08:10:34
Number of particles: 	1
Selected integrator: 	ias15
Simulation time:     	0.000000
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, id=-1 m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
---------------------------------


The reference plane is the ecliptic (Earth's orbital plane) of J2000 (Jan. 1st 2000 12:00 GMT), with the x-axis along the ascending node of the ecliptic and the Earth's mean equator (also at J2000).  